# Estudos com o DeepSeek R1

Para esse estudo foi instalado os seguintes softwares:
- [Ollama](https://ollama.com/)
- [deepseek-r1:7b](https://ollama.com/library/deepseek-r1) -- Prefiro as respostas do modelo destilado do Alibaba.

Aqui no caderno, eu utilizo apenas o framework do Ollama em Python:
```sh
pip install ollama
```

Eu preferi não formatar a resposta, porque acaba sumindo a parte do pensamento `<think>...</think>`.

## Extração de dados

In [27]:
from ollama import chat

response = chat(
    model="deepseek-r1:7b",
    messages=[
        {
            "role": "user",
            "content": "Tell me about Canada.",
        }
    ],
    stream=True,
)

for chunk in response:
    print(chunk.message.content, end="")

<think>

</think>

Canada is a country located in northern North America, known for its vast natural landscapes and rich cultural heritage. Here are some key points about Canada:

### Geography
- **Location**: Canada is situated on the northern part of the Americas, bordered by the United States to the south and the Arctic Ocean to the north.
- **Size**: It is the second-largest country in the world by both land area and total area.
- **Continents**: Canada covers parts of three continents: Europe (in the northeast), Asia (in the east), and Africa (in the central and northern regions).

### Capitals
- **Official Capital**: Ottawa, capital since 1980.
- **Political Capital**: also Ottawa.

### Official Languages
- English is the official language, with over 35 First Nations languages recognized.

### Government
- **Political System**: A constitutional monarchy with a federal system of government. The Prime Minister of Canada is the head of government and a member of theRoyal Club.
- **B

In [28]:
from ollama import chat


response = chat(
    model="deepseek-r1:7b",
    messages=[
        {
            "role": "user",
            "content": "Conte-me sobre o Canada.",
        }
    ],
    stream=True,
)

for chunk in response:
    print(chunk.message.content, end="")

<think>
Alright, the user asked for information about Canada in Portuguese. I need to make sure my response is both informative and engaging.

First, I'll start by introducing Canada as a significant country in the world. It's important to mention its vastness with over 10 million square kilometers and its strategic location on the northern hemisphere.

Next, I should highlight some of the main provinces or territories: Ontario, Quebec, British Columbia, Alberta, Saskatchewan, Manitoba, New Brunswick, Nova Scotia, Newfoundland and Labrador, and Prince Edward Island. Including the tenEncode project might be a good idea for cultural insights.

I'll also touch on Canada's official languages—English is predominant in business and government, while French is used in Quebec and recognized elsewhere. This shows diversity within the country.

Economically, I should mention sectors like energy, mining, manufacturing, and technology. The economy is indeed robust, so emphasizing growth areas woul

## Extração de dados via Pydantic

In [29]:
from ollama import chat
from pydantic import BaseModel


class Country(BaseModel):
    name: str
    think: str
    capital: str
    languages: list[str]


response = chat(
    messages=[
        {
            "role": "user",
            "content": "Tell me about Canada.",
        }
    ],
    model="deepseek-r1:7b",
    format=Country.model_json_schema(),
)

country = Country.model_validate_json(response.message.content)
print(f"{country=}")

country=Country(name='Canada', think='Canada is a country in the northern part of the Americas, known for its vast natural landscapes and multicultural society.', capital='Ottawa', languages=['English', 'French', 'Spanish'])


In [30]:
from ollama import chat
from pydantic import BaseModel


class Pais(BaseModel):
    nome: str
    capital: str
    idiomas: list[str]


response = chat(
    messages=[
        {
            "role": "user",
            "content": "Conte-me sobre o Canada.",
        }
    ],
    model="deepseek-r1:7b",
    format=Pais.model_json_schema(),
)

country = Pais.model_validate_json(response.message.content)
print(f"{country=}")

country=Pais(nome='Canada', capital='Ottawa', idiomas=['ingles', 'polski', 'espanhol', 'francês'])


## Conversação

In [4]:
from ollama import chat
from io import StringIO


class DeepSeekChat:

    def __init__(self, model="deepseek-r1:7b"):
        self._model = model
        self._messages = [
            {
                "role": "system",
                "content": "Eu sou uma assistente virtual para brasileiros do Brasil, todas as respostas serão em português do Brasil",
            }
        ]

    def send(self, question):
        self._messages.append(
            {
                "role": "user",
                "content": question,
            }
        )
        response = chat(
            model=self._model,
            messages=self._messages,
            stream=True,
        )
        message_io = StringIO()
        for chunk in response:
            message_io.write(chunk_msg := chunk.message.content)
            yield chunk_msg
        self._messages.append(
            {
                "role": "assistant",
                "content": message_io.getvalue(),
            }
        )

    def __repr__(self):
        return f"<{self.__class__.__name__} model='{self._model}'>"


deepseek_chat = DeepSeekChat()
deepseek_chat

<DeepSeekChat model='deepseek-r1:7b' 129914574794448>

In [3]:
question = "Conte-me sobre o país Canada"
for chunk in deepseek_chat.send(question):
    print(chunk, end="")

<think>
Okay, the user wants information about Canada in Portuguese. They mentioned they're a virtual assistant for Brazilian users, so I should respond in Brazilian Portuguese.

I'll start by greeting them and then provide an overview of Canada's culture, geography, economy, and history. Maybe mention its location between the US and Mexico to give context.

I should highlight the diverse cities like Toronto, Vancouver, Montreal, and Calgary to show its multicultural aspect. Also, touch on the multicultural vibe in smaller towns where people from different backgrounds live together.

Economically, Canada has strong sectors like oil and gas but is also known for innovation and tech companies. Including some famous cities will make it relatable.

I'll talk about its natural beauty with forests, mountains, and big cities near the coast to show the country's scenic spots.

Including cultural aspects like English and French influences would be good, as well as First Nations culture to respe

In [51]:
question = "O Canada é um país de esquerda ou de direita"
for chunk in deepseek_chat.send(question):
    print(chunk, end="")

<think>
Okay, the user is asking if Canada is a left or right-leaning country. I need to provide an accurate answer in Brazilian Portuguese.

First, I should explain that Canada isn't leaning left or right because it's a federal democracy with multiple provinces and territories. Each has their own political landscape.

I'll mention that thefederal government itself doesn't have a primary party, but provinces like Ontario and British Columbia tend to lean left due to high taxes and population density. In contrast, Prairie provinces often lean right for similar reasons.

It's also good to note that overall, Canadian politics are centrist, with more than half of people supporting progressive policies and a significant portion disliking austerity measures.

I should keep the response clear and concise, avoiding any technical jargon so it's easy to understand.
</think>

O Canadá não é um país de esquerda ou de direita, pois é uma demografia federada e polêmica. No Canadá, não há single part

![captura de processamento](./files/img/2025/02/02/captura_tela_2025-02-02_14-56-50.png)

Processamento do é bem tranquilo.